In [1]:
import pandas as pd
import numpy as np

In [2]:
## Create dataframe
df = pd.read_csv('books.csv',encoding="latin",header=None)
df.columns = ['img_idx','img_file','img_link','title','author','cat_id','category']
df.head()

,img_idx,img_file,img_link,title,author,cat_id,category
0,761183272,0761183272.jpg,http://ecx.images-amazon.com/images/I/61Y5cOdH...,Mom's Family Wall Calendar 2016,Sandra Boynton,3,Calendars
1,1623439671,1623439671.jpg,http://ecx.images-amazon.com/images/I/61t-hrSw...,Doug the Pug 2016 Wall Calendar,Doug the Pug,3,Calendars
2,B00O80WC6I,B00O80WC6I.jpg,http://ecx.images-amazon.com/images/I/41X-KQqs...,"Moleskine 2016 Weekly Notebook, 12M, Large, Bl...",Moleskine,3,Calendars
3,761182187,0761182187.jpg,http://ecx.images-amazon.com/images/I/61j-4gxJ...,365 Cats Color Page-A-Day Calendar 2016,Workman Publishing,3,Calendars
4,1578052084,1578052084.jpg,http://ecx.images-amazon.com/images/I/51Ry4Tsq...,Sierra Club Engagement Calendar 2016,Sierra Club,3,Calendars


In [3]:
#What are our categories?
df.category.unique()

array(['Calendars', 'Comics & Graphic Novels', 'Test Preparation',
       'Mystery, Thriller & Suspense', 'Science Fiction & Fantasy',
       'Romance', 'Humor & Entertainment', 'Literature & Fiction',
       'Gay & Lesbian', 'Engineering & Transportation',
       'Cookbooks, Food & Wine', 'Crafts, Hobbies & Home',
       'Arts & Photography', 'Education & Teaching',
       'Parenting & Relationships', 'Self-Help', 'Computers & Technology',
       'Medical Books', 'Science & Math', 'Health, Fitness & Dieting',
       'Business & Money', 'Law', 'Biographies & Memoirs', 'History',
       'Politics & Social Sciences', 'Reference',
       'Christian Books & Bibles', 'Religion & Spirituality',
       'Sports & Outdoors', 'Teen & Young Adult', "Children's Books",
       'Travel'], dtype=object)

In [4]:
import cleaner
cleaner.build_images_parallel(df, 25)

In [5]:
#test train split the dataframe
from sklearn.model_selection import train_test_split
#Notice that we don't drop target from x_train, since we have to feed a dataframe with target
# to train_generator

target = df['category']
x_train, x_test, y_train, y_test = train_test_split(df, target, test_size=0.2)


#have to reset index of x_test so that can use image_data_generator on it ??
x_test = x_test.reset_index()
y_test = y_test.reset_index()
print(x_test.head())

    index     img_idx        img_file  \
0  110861   989167615  0989167615.jpg   
1   59141  1565238133  1565238133.jpg   
2  148860   849947227  0849947227.jpg   
3  195203  009174153X  009174153X.jpg   
4    5678  1455703338  1455703338.jpg   

                                            img_link  \
0  http://ecx.images-amazon.com/images/I/51X3DJhC...   
1  http://ecx.images-amazon.com/images/I/51n%2BQZ...   
2  http://ecx.images-amazon.com/images/I/41jz2Snm...   
3  http://ecx.images-amazon.com/images/I/51csIjCf...   
4  http://ecx.images-amazon.com/images/I/51MRO9gG...   

                                               title  \
0  A Just Cause: A True Story of Courage, Hope, &...   
1  Great Book of Tattoo Designs, Revised Edition:...   
2  Grieving God's Way: The Path to Lasting Hope a...   
3  The Wilder Shores of Marx: Journeys in a Vanis...   
4               Admission Assessment Exam Review, 3e   

                     author  cat_id                  category  
0  James H. Hol

In [6]:
from keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

directory = 'pictures/'
batch_size = 32
train_generator = ImageDataGenerator(rescale=1./255).flow_from_dataframe(x_train, directory, x_col='img_file', y_col='category', has_ext=True, target_size=(256, 256), color_mode='rgb', classes=None, class_mode='categorical', batch_size=batch_size, shuffle=True, seed=None, save_to_dir=None, save_prefix='', save_format='jpeg', subset=None, interpolation='nearest')
#train_images, train_labels = next(train_generator)

test_generator = ImageDataGenerator(rescale=1./255).flow_from_dataframe(x_test, directory, x_col='img_file', y_col='category', has_ext=True, target_size=(256, 256), color_mode='rgb', classes=None, class_mode='categorical', batch_size=batch_size, shuffle=True, seed=None, save_to_dir=None, save_prefix='', save_format='jpeg', subset=None, interpolation='nearest')



/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 154543 images belonging to 32 classes.
Found 38592 images belonging to 32 classes.


In [7]:
#do not reshape

#print('Before: ', train_images.shape)
#train_images = train_images.reshape(train_images.shape[0], -1)
#print('After: ', train_images.shape)
# That's a lot of input neurons!

In [8]:
num_classes = len(df.category.unique())

In [9]:
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [10]:
from keras.applications import inception_v3

base_model=inception_v3.InceptionV3(weights='imagenet',include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# fully-connected layer with 1024 neurons 
x = Dense(1024, activation='relu')(x)
# and a logistic layer
predictions = Dense(num_classes, activation='softmax')(x)
# this is the model we will train
model = Model(input=base_model.input, output=predictions)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if sys.path[0] == '':


In [11]:
for layer in model.layers:
    print(layer.name)

input_1
conv2d_1
batch_normalization_1
activation_1
conv2d_2
batch_normalization_2
activation_2
conv2d_3
batch_normalization_3
activation_3
max_pooling2d_1
conv2d_4
batch_normalization_4
activation_4
conv2d_5
batch_normalization_5
activation_5
max_pooling2d_2
conv2d_9
batch_normalization_9
activation_9
conv2d_7
conv2d_10
batch_normalization_7
batch_normalization_10
activation_7
activation_10
average_pooling2d_1
conv2d_6
conv2d_8
conv2d_11
conv2d_12
batch_normalization_6
batch_normalization_8
batch_normalization_11
batch_normalization_12
activation_6
activation_8
activation_11
activation_12
mixed0
conv2d_16
batch_normalization_16
activation_16
conv2d_14
conv2d_17
batch_normalization_14
batch_normalization_17
activation_14
activation_17
average_pooling2d_2
conv2d_13
conv2d_15
conv2d_18
conv2d_19
batch_normalization_13
batch_normalization_15
batch_normalization_18
batch_normalization_19
activation_13
activation_15
activation_18
activation_19
mixed1
conv2d_23
batch_normalization_23
activat

In [ ]:
#train this biiiitch
EPOCHS = 1

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

# need to fit by generator since data won't fit into memory!
#model.fit(x_train,
#          y_train,
#          epochs=10,
#          batch_size=32,
#          validation_data=(x_test, y_test))

model.fit_generator(train_generator, validation_data=test_generator, 
                    steps_per_epoch=len(x_train)//batch_size, 
                    validation_steps=len(x_test)//batch_size, epochs=EPOCHS)

#ok
#ok

Epoch 1/1
  15/5189 [..............................] - ETA: 25:18:08 - loss: 0.1520 - acc: 0.9667

In [ ]:
#TODO: get live training vs validation loss graph... and speed this up somehow! 